In [1]:
def _accuracy(labels, predictions):
    equal = tf.equal(tf.cast(labels, tf.float32), predictions)
    equal_float = tf.cast(equal, tf.float32)
    return tf.reduce_mean(equal_float)

def fit_batch(self, x, y):
    graph = tf.Graph()

    with graph.as_default():
        # w = tf.Variable(tf.random_normal([3, 1]), dtype=tf.float32)
        w = tf.Variable(tf.zeros([3, 1]), dtype=tf.float32)
        # x_ = np.hstack([x, np.ones([len(x), 1], dtype=np.float32)])
        x_ = np.c_[x, np.ones(len(x), dtype=np.float32)]
        y_hat = tf.sign(tf.matmul(x_, w))
        # y_hat = self._step(tf.matmul(x_, w))
        error = y - y_hat
        mse = tf.reduce_mean(tf.square(error))
        y_tensor = tf.convert_to_tensor(y)
        print(y_tensor.shape)
        print(y_hat.shape)
        print('=' * 10)
        # accuracy, acc_op = tf.metrics.accuracy(labels=y_tensor, predictions=y_hat)
        accuracy = self._accuracy(y_tensor, y_hat)
        delta = tf.matmul(x_, error, transpose_a=True)
        train = tf.assign(w, tf.add(w, self.learning_rate * delta))

    with tf.Session(graph=graph) as sess:
        init = tf.global_variables_initializer()
        # init_local = tf.local_variables_initializer()
        # sess.run([init, init_local])
        sess.run(init)

        for epoch in range(self.epochs):
            error_, w_, acc_, yh, yt = sess.run([mse, train, accuracy, y_hat, y_tensor])
            # error_, w_ = sess.run([mse, train])
            print('Epoch: {} MSE: {}, w: {}, acc: {}, yhat: {}'.format(epoch + 1,
                                                             error_,
                                                             w_.reshape(1, len(w_)),
                                                             acc_, yh.reshape(1, len(w_) + 1)
                                                            ))

In [ ]:
# customer activation

from tensorflow.python.framework import ops

@staticmethod
def _step(x):
    def _step_actual(x):
        return 1 if x > 0 else 0

    np_step = np.vectorize(_step_actual)

    np_ = lambda x: np_step(x).astype(np.float32)
    with ops.op_scope([x], 'step', 'step') as name:
        y = tf.py_func(np_, [x], [tf.float32], name='step', stateful=False)
        return y[0]